In [2]:
import os

In [3]:
%pwd

'c:\\Users\\HP\\Documents\\projects\\Chicken-Disease-Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\HP\\Documents\\projects\\Chicken-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


# decorator is used to automatically generate special methods for the class based on its defined fields
@dataclass(frozen=True) #  frozen=True parameter makes the class immutable, meaning its fields cannot be modified after instantiation
class DataIngestionConfig: #  define the name of the data class
    root_dir: Path # field of the class named root_dir of type Path from the pathlib module. It represents the root directory for the data ingestion stage
    source_URL: str # field of the class named source_URL of type str. It represents the URL from which the data for ingestion will be sourced.
    local_data_file: Path # field of the class named local_data_file of type Path. It represents the path where the downloaded data file will be stored locally.
    unzip_dir: Path # field of the class named unzip_dir of type Path. It represents the directory where the downloaded zip file will be extracted/unzipped.

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager: # define the name of the class
    def __init__( # constructor method for the ConfigurationManager class
        self,
        # takes two optional parameters: config_filepath and params_filepath
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # read the YAML file specified by config_filepath and assign the contents to the self.config attribute. The read_yaml function is assumed to be defined elsewhere and is responsible for parsing the YAML file.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # call a function create_directories and pass a list containing the artifacts_root attribute from the self.config object. This function is be defined elsewhere and is responsible for creating the specified directories.

        create_directories([self.config.artifacts_root])


    
    # This method returns an instance of DataIngestionConfig, which is a data class representing the configuration for data ingestion.
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion # retrieve the data_ingestion attribute from the self.config object and assigns it to the config variable.

        create_directories([config.root_dir]) # call the create_directories function to create the directory specified by config.root_dir.


        # creates an instance of DataIngestionConfig using the values from the config object. This instance represents the configuration for the data ingestion stage.
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config # eturns the data_ingestion_config object

In [11]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion: # defines the name of the class
    def __init__(self, config: DataIngestionConfig): # constructor method for the DataIngestion class. It takes a single parameter config of type DataIngestionConfig
        self.config = config # assigns the config parameter to the self.config attribute, making it accessible within the class.


    
    # This method downloads the data file specified in the configuration.
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): #  check if the local data file specified in the configuration does not already exist.
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, # urlretrieve function from the request module is used to download the file specified by the source_URL attribute in the configuration
                filename = self.config.local_data_file # The downloaded file is saved with the path specified by local_data_file.
            )

            # It logs an information message stating that the file has been downloaded, along with additional information such as the file name and headers.
            logger.info(f"{filename} download! with following info: \n{headers}")

        #  Else it logs an information message stating that the file already exists, along with its size    
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    # This method extracts a zip file into the data directory specified in the configuration.
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir # assign the unzip_dir attribute from the configuration to the unzip_path variable.
        os.makedirs(unzip_path, exist_ok=True) # creates the directory specified by unzip_path if it doesn't already exist.

        # opens the zip file specified by local_data_file attribute in the configuration for reading.
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) # extracts all the contents of the zip file into the unzip_path directory.

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-21 14:45:58,629: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-21 14:45:58,648: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-21 14:45:58,653: INFO: common: created directory at: artifacts]
[2023-06-21 14:45:58,656: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-21 14:46:12,064: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C4FC:93C9:420FA1:4E64A8:6492BFD2
Accept-Ranges: bytes
Date: Wed, 21 Jun 2023 09:16:05 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10245-MAA
X-Cache: MI